<a href="https://colab.research.google.com/github/thiago2608santana/Natural_Language_Processing_with_Python/blob/main/Chat_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importar bibliotecas

In [41]:
import pickle
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Carregar os dados

In [2]:
with open('/content/drive/MyDrive/NLP_with_Python_Datasets/train_qa.txt', 'rb') as f:
  train_data = pickle.load(f)

In [3]:
with open('/content/drive/MyDrive/NLP_with_Python_Datasets/test_qa.txt', 'rb') as f:
  test_data = pickle.load(f)

# Visualizar e entender os dados

In [4]:
type(train_data)

list

In [5]:
type(test_data)

list

In [6]:
len(train_data)

10000

In [7]:
len(test_data)

1000

**story**

In [11]:
' '.join(train_data[0][0])

'Mary moved to the bathroom . Sandra journeyed to the bedroom .'

**question**

In [12]:
' '.join(train_data[0][1])

'Is Sandra in the hallway ?'

**answer**

In [24]:
train_data[0][2]

'no'

# Preprocessamento dos dados

In [16]:
all_data = test_data + train_data

In [17]:
len(all_data)

11000

In [18]:
vocab = set()

for story, question, answer in all_data:
  vocab = vocab.union(set(story))
  vocab = vocab.union(set(question))

In [25]:
vocab.add('no')

In [26]:
vocab.add('yes')

In [27]:
vocab

{'.',
 '?',
 'Daniel',
 'Is',
 'John',
 'Mary',
 'Sandra',
 'apple',
 'back',
 'bathroom',
 'bedroom',
 'discarded',
 'down',
 'dropped',
 'football',
 'garden',
 'got',
 'grabbed',
 'hallway',
 'in',
 'journeyed',
 'kitchen',
 'left',
 'milk',
 'moved',
 'no',
 'office',
 'picked',
 'put',
 'the',
 'there',
 'to',
 'took',
 'travelled',
 'up',
 'went',
 'yes'}

In [28]:
vocab_len = len(vocab) + 1

In [29]:
vocab_len

38

**Longest story**

In [31]:
all_story_lens = [len(data[0]) for data in all_data]

In [32]:
max_story_len = max(all_story_lens)

In [33]:
max_story_len

156

**Longets question**

In [38]:
max_question_len = max([len(data[1]) for data in all_data])

In [40]:
max_question_len

6

# Tokenizar o dataset

In [42]:
tokenizer = Tokenizer(filters=[])

In [43]:
tokenizer.fit_on_texts(vocab)

In [44]:
tokenizer.word_index

{'kitchen': 1,
 'moved': 2,
 'is': 3,
 'went': 4,
 'bedroom': 5,
 'daniel': 6,
 'grabbed': 7,
 'dropped': 8,
 'picked': 9,
 'mary': 10,
 'the': 11,
 'put': 12,
 'john': 13,
 '?': 14,
 'down': 15,
 'garden': 16,
 'hallway': 17,
 'office': 18,
 '.': 19,
 'football': 20,
 'back': 21,
 'bathroom': 22,
 'to': 23,
 'got': 24,
 'yes': 25,
 'there': 26,
 'in': 27,
 'milk': 28,
 'took': 29,
 'apple': 30,
 'travelled': 31,
 'left': 32,
 'up': 33,
 'no': 34,
 'sandra': 35,
 'journeyed': 36,
 'discarded': 37}

In [45]:
train_story_text = []
train_question_text = []
train_answers = []

for story, question, answer in train_data:
  train_story_text.append(story)
  train_question_text.append(question)
  train_answers.append(answer)

In [46]:
train_story_seq = tokenizer.texts_to_sequences(train_story_text)

# Função que vetoriza os dados (story, question, answer)

In [48]:
def vectorize_stories(data, word_index=tokenizer.word_index, max_story_len=max_story_len, max_question_len=max_question_len):

  #Stories = X
  X = []
  #Questions = Xq
  Xq = []
  #Correct answer (yes/no) = Y
  Y = []

  for story, query, answer in data:

    x = [word_index[word.lower()] for word in story]
    xq = [word_index[word.lower()] for word in query]

    y = np.zeros(len(word_index)+1)
    y[word_index[answer]] = 1

    X.append(x)
    Xq.append(xq)
    Y.append(y)

  return (pad_sequences(X, maxlen=max_story_len), pad_sequences(Xq, maxlen=max_question_len), np.array(Y))

# Vetorizar os dados

In [51]:
inputs_train, queries_train, answers_train = vectorize_stories(train_data)

In [52]:
inputs_test, queries_test, answers_test = vectorize_stories(test_data)

In [55]:
inputs_test

array([[ 0,  0,  0, ..., 11,  5, 19],
       [ 0,  0,  0, ..., 11, 16, 19],
       [ 0,  0,  0, ..., 11, 16, 19],
       ...,
       [ 0,  0,  0, ..., 11, 30, 19],
       [ 0,  0,  0, ..., 11, 16, 19],
       [ 0,  0,  0, ..., 30, 26, 19]], dtype=int32)

In [56]:
answers_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

**Verificar a quantidade de yes/no e checar a posição (index) correspondente. Tudo ok**

In [57]:
tokenizer.word_index['yes']

25

In [58]:
tokenizer.word_index['no']

34

In [59]:
sum(answers_test)

array([  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0., 497.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0., 503.,   0.,   0.,   0.])